In [115]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv2

# Import MINST data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# normalization
x_train = x_train / 255.0
x_test = x_test / 255.0

In [116]:
# Parameters
learning_rate = 0.005
training_epochs = 4
batch_size = 512
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.85 # исключение Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])), # 5x5 conv, 1 input, 32 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])), # 5x5 conv, 32 inputs, 64 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])), # fully connected, 7*7*64 inputs, 1024 outputs
    'out': tf.Variable(tf.random_normal([1024, n_classes])) # 1024 inputs, 10 outputs (class prediction)
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes])) #свободные веса
}

In [117]:
# reshape MINST data
ds_train = x_train.reshape(x_train.shape[0], n_input)
ds_test = x_test.reshape(x_test.shape[0], n_input)
label_test=np.zeros(shape=(y_test.shape[0],n_classes),dtype="float32")
for k in range(y_test.shape[0]):
  label_test[k][y_test[k]]=1.0

In [118]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [119]:
# Launch the graph
def run(learning_rate, training_epochs, batch_size, display_step, ds_test, label_test):
   # Construct model
    pred = conv_net(x, weights, biases, keep_prob)

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initializing the variables
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)

        step = 1
        batch_xs=np.zeros(shape=(batch_size,784),dtype="float32")
        
        # Training cycle
        for epoch in range(training_epochs):
            batch_ys=np.zeros(shape=(batch_size,n_classes),dtype="float32")
            total_batch = int(ds_train.shape[0]/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            batch_ys=np.zeros(shape=(batch_size,n_classes),dtype="float32")
            for k in range(batch_size):
                index = k+batch_size*i
                batch_xs[k] = ds_train[index] 
                batch_ys[k][y_train[index]]=1.0
            
            # Run optimization op (backprop)
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
            
            if step != 0 and step % display_step == 0:
                # Calculate batch loss and accuracy
                loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
                print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                    "{:.6f}".format(loss) + ", Training Accuracy= " + \
                    "{:.5f}".format(acc))
            step += 1
        #print("Optimization Finished!")
        
        # Calculate accuracy for 256 mnist test images
        """        print("Testing Accuracy:", \
            sess.run(accuracy, feed_dict={x: ds_test[:256],
                                        y: label_test[:256],
                                        keep_prob: 1.}))
        """
        return sess.run(accuracy, feed_dict={x: ds_test,
                                        y: label_test,
                                        keep_prob: 1.})


In [120]:
run(learning_rate, training_epochs, batch_size, display_step, ds_test, label_test)

Iter 5120, Minibatch Loss= 30280.326172, Training Accuracy= 0.69336
Iter 10240, Minibatch Loss= 7466.995117, Training Accuracy= 0.73242
Iter 15360, Minibatch Loss= 2734.855957, Training Accuracy= 0.86133
Iter 20480, Minibatch Loss= 1790.861816, Training Accuracy= 0.91406
Iter 25600, Minibatch Loss= 1515.815186, Training Accuracy= 0.89844
Iter 30720, Minibatch Loss= 1226.599121, Training Accuracy= 0.89844
Iter 35840, Minibatch Loss= 712.312256, Training Accuracy= 0.92969
Iter 40960, Minibatch Loss= 738.901184, Training Accuracy= 0.94922
Iter 46080, Minibatch Loss= 1177.577148, Training Accuracy= 0.89844
Iter 51200, Minibatch Loss= 388.500000, Training Accuracy= 0.94727
Iter 56320, Minibatch Loss= 265.766693, Training Accuracy= 0.96680


0.9469

In [122]:
display_step = 10000
for q in range(0, 10):
    image = cv2.imread('nm'+str(q)+'.png', cv2.IMREAD_GRAYSCALE)
    image = [*(image / 255.0).flatten()]
    image = [abs(x - 1) for x in image]
    label_test = [0] * 10
    label_test[q] = 1
    res = run(learning_rate, training_epochs, batch_size, display_step, [image], [label_test])
    print('Number ', 'n'+str(q), ' was understood: ', res)
    

Number  n0  was understood:  1.0
Number  n1  was understood:  0.0
Number  n2  was understood:  1.0
Number  n3  was understood:  1.0
Number  n4  was understood:  1.0
Number  n5  was understood:  1.0
Number  n6  was understood:  0.0
Number  n7  was understood:  0.0
Number  n8  was understood:  1.0
Number  n9  was understood:  0.0
